In [ ]:
!pip install ax-platform

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 48.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import tempfile
import pickle
from pathlib import Path

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize

In [ ]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render
from ax.utils.tutorials.cnn_utils import evaluate, load_mnist, train

[WARNING 09-30 09:51:17] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading data

In [ ]:

# from scipy.fft import dst

class SlidingWindowDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        f = self.data[idx].reshape(10, 10)

        return f # Reshape to (10, 1) to verify for our new data

In [ ]:
def data_loading(data_path, batch_size):

    data = np.load(data_path)

    # Normalize only the RGB features (last 5 features)
    # Compute mean and std for RGB features across all samples and time steps
    rgb_features = data[:, :, 5:]
    rgb_mean = np.mean(rgb_features, axis=(0, 1))
    rgb_std = np.std(rgb_features, axis=(0, 1))
    # Avoid division by zero
    rgb_std[rgb_std == 0] = 1.0

    # Normalize the RGB features
    data[:, :, 5:] = (data[:, :, 5:] - rgb_mean) / rgb_std

    # NOTE: Optional, maybe for later: Scale the normalized RGB features to match the scale of motion features
    # For example, you can scale them to have the same standard deviation as the motion features
    motion_features = data[:, :, :5]
    motion_std = np.std(motion_features, axis=(0, 1))
    scaling_factor = motion_std.mean()
    data[:, :, 5:] *= scaling_factor

    print(data[215080:265080].shape)
    # Create the dataset
    dataset = SlidingWindowDataset(data[215080:265080])

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    # Split the dataset
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Set up data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader


In [ ]:
data_path = '/content/drive/MyDrive/Dataset_d14_d15/combined_d14_d15.npy'
batch_size = 2048
train_loader,val_loader = data_loading(data_path,batch_size)

(50000, 10, 10)


### Model Loading

In [ ]:
class Encoder_Decoder(nn.Module):
  def __init__(self,in_size,hidden_size,no_layers):
    super(Encoder_Decoder,self).__init__()
    ### layer1 is the encoder part, which is bidirectional

    self.layer1 = nn.GRU(input_size=in_size,
                                         hidden_size=hidden_size//2,
                                         num_layers=no_layers,
                                         batch_first=True,
                                         bidirectional=True)
    ### layer 2 is decoder unidirectional we are doing fixed state
    self.layer2 = nn.GRU(input_size=hidden_size,hidden_size=hidden_size,num_layers=1,batch_first=True)

    self.layer3 = nn.Sequential(
                                nn.Tanh(),
                                nn.Linear(hidden_size,in_size) ) # adding batchnormalization at the end

    # self.constant_hidden = constant_hidden.unsqueeze(0).unsqueeze(0)
  def forward(self,x):
    # x = nn.BatchNorm1d(x.shape[1]).to(device)(x)
    # x = nn.BatchNorm1d(x.shape[1])
    x,_ = self.layer1(x) # output is hideen state
    # print(x.shape)
    outputs = []
    input_vector = torch.zeros_like(x[:,0,:]).unsqueeze(dim=1)
    # cell_state = torch.zeros_like(x[:,0,:]).unsqueeze(dim=0)
    # print(input_vector.shape)
    # print( x[:,-1,:].unsqueeze(1).shape)
    for t in range(x.size(1)):
            # print( x[:,-1,:].squeeze(1).unsqueeze(0).shape,input_vector.shape)
            # Perform GRU step with constant hidden state
            # out, (hidden_state, cell_state) = self.layer2(input_vector, (x[:,-1,:].squeeze(1).unsqueeze(0), cell_state))
            out, _ = self.layer2(input_vector # current state
                                 , x[:,-1,:].squeeze(1).unsqueeze(0) # last hidden state
                                 )
            # print(out.shape)
            # print(out.shape)
            # Pass the GRU output through a fully connected layer
            # print(out.shape)
            prediction = self.layer3(out.squeeze(1)) # fc is to get back to pose dimension

            # Store the prediction and use it as the input for the next step
            outputs.append(prediction)
            input_vector = out


        # Stack all the outputs to form the final output sequence
    outputs = torch.stack(outputs, dim=1)
        # return outputs
    # x,_ = self.layer2(x)
    return outputs

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
in_size = 1
hidden_size = 32
no_layers=2 ## number of bi grus in encoder
enc_dec_model = Encoder_Decoder(in_size,hidden_size,no_layers)
enc_dec_model = enc_dec_model.to(device)

In [ ]:

count = 0
count+=(sum(param.numel() for param in enc_dec_model.parameters() if param.requires_grad))
print(f"Number of parameters in the Encoder Decoder model {count}")

Number of parameters in the Encoder Decoder model 12993


### Bayesian_optimization

In [ ]:
def net_train(net, train_loader, parameters, dtype, device):
  net.to( device=device)

  # Define loss and optimizer
  criterion = nn.MSELoss()
  # optimizer = optim.SGD(net.parameters(), # or any optimizer you prefer
  #                       lr=parameters.get("lr", 0.001), # 0.001 is used if no lr is specified
  #                       momentum=parameters.get("momentum", 0.9)
  # )
  optimizer = optim.Adam(net.parameters(), lr=parameters.get('lr',0.001),
                         weight_decay=parameters.get("weight_decay",0.00039147384191798724))
  scheduler = ReduceLROnPlateau(optimizer, 'min',
                                patience=parameters.get("patience",5),
                                factor=0.5)
  # scheduler = optim.lr_scheduler.StepLR(
  #     optimizer,
  #     step_size=int(parameters.get("step_size", 30)),
  #     gamma=parameters.get("gamma", 1.0),  # default is no learning rate decay
  # )
  best_val_loss = float('inf')
  num_epochs = 15 # Play around with epoch number

  net.train()
  # Train Network
  for _ in range(num_epochs):

      for inputs in train_loader:
          # move data to proper dtype and device
          inputs = inputs.to(dtype=dtype, device=device)

          # zero the parameter gradient
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(inputs)
          loss = criterion(outputs, inputs)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(net.parameters(), parameters.get("clip_value", 10.47314254460227))
          optimizer.step()
      val_loss = 0
      # Evaluate Network
      for inputs in val_loader:
          inputs = inputs.to(dtype=dtype, device=device)
          outputs = net(inputs)
          loss = criterion(outputs, inputs)
          val_loss += loss.item()
          # if loss.item() < best_val_loss:
          #     best_val_loss = loss.item()
          #     torch.save(net.state_dict(), 'best_model.pth')

      scheduler.step(val_loss)
  return net

In [ ]:
def init_net(parameterization):

    model = Encoder_Decoder(10,parameterization.get("hidden_dim",512),parameterization.get("no_layers",5))

    print("Number of parameters to train: {}".format(sum(p.numel() for p in model.parameters())))
    return model # return untrained model

In [ ]:
def evaluate(net, data_loader, dtype, device, parametrization): # changed model to net, val_loader to data_loader
    net.eval()
    total_loss = 0
    criterion = nn.MSELoss() #Moved criterion outside of the loop
    with torch.no_grad():
        for batch in data_loader: # Changed batch to batch,_ to fit the expected output from train_loader
            batch = batch.to(dtype=dtype, device=device) #Added dtype and device
            outputs = net(batch)
            loss = criterion(outputs, batch)  # Changed inputs to batch
            total_loss += loss.item()
    # parametrization.get("loss", total_loss / len(data_loader)) # Removed - Incorrect behavior
    return total_loss / len(data_loader) # Return the calculated loss

In [ ]:
def train_evaluate(parameterization):

    # constructing a new training data loader allows us to tune the batch size
    # train_loader = torch.utils.data.DataLoader(train_set,
    #                             batch_size=parameterization.get("batchsize", 32),
    #                             shuffle=True,
    #                             num_workers=1,
    #                             pin_memory=True)

    # Get neural net
    untrained_net = init_net(parameterization)

    # train
    trained_net = net_train(net=untrained_net, train_loader=train_loader,
                            parameters=parameterization, dtype=dtype, device=device)

    # return the accuracy of the model as it was trained in this run
    loss = evaluate(
        net=trained_net,
        data_loader=val_loader,
        dtype=dtype,
        device=device,
        parametrization=parameterization
    )

    return {"loss": loss} # Return the loss as a dictionary for AxClient

In [ ]:

# best_parameters, values, experiment, model = optimize
ax_client = AxClient()

[INFO 09-30 09:51:42] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [ ]:
#torch.cuda.set_device(0) #this is sometimes necessary for me
from ax.modelbridge.registry import Models
dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ax_client.create_experiment(
    name='hyper_parameters_tuning',
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4],'value_type':"float", "log_scale": True},
        {"name": "hidden_dim", "type": "choice", "values": [256,512,1024],'value_type':"int"},
        {"name": "no_layers", "type": "range", "bounds": [3, 5],'value_type':"int"},
        {"name": "weight_decay", "type": "range", "bounds": [1e-5, 0.001],'value_type':"float"},
        {"name": "patience", "type": "range", "bounds": [1, 10],'value_type':"int"},
        # {"name": "num_epochs", "type": "range", "bounds": [10, 100],'value_type':"int"},
        {"name": "clip_value", "type": "range", "bounds": [5.1, 100.0],'value_type':"float"},
        #{"name": "max_epoch", "type": "range", "bounds": [1, 30]},
        #{"name": "stepsize", "type": "range", "bounds": [20, 40]},
    ],
    objectives={"loss": ObjectiveProperties(minimize=True)},
    overwrite_existing_experiment=True
    # evaluation_function=train_evaluate,

    # objective_name='loss',
    # minimize=True,
    # experiment_name='hyper_parameters_tuning',
    # total_trials=10
)

# print(best_parameters)
# means, covariances = values
# print(means)
# print(covariances)

/usr/local/lib/python3.10/dist-packages/ax/service/utils/instantiation.py:248: AxParameterWarning:

`is_ordered` is not specified for `ChoiceParameter` "hidden_dim". Defaulting to `True`  since the parameter is not of type string.. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction. Note that choice parameters with exactly 2 choices are always considered ordered and that the user-supplied `is_ordered` has no effect in this particular case.

/usr/local/lib/python3.10/dist-packages/ax/service/utils/instantiation.py:248: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "hidden_dim". Defaulting to `True` for parameters of `ParameterType` INT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 09-30 09:51:46] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='lr', parameter_type=FLOAT, range=[1

In [ ]:
for i in range(50):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))

/usr/local/lib/python3.10/dist-packages/ax/modelbridge/cross_validation.py:463: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 09-30 09:52:07] ax.service.ax_client: Generated new trial 0 with parameters {'lr': 0.013386, 'hidden_dim': 256, 'no_layers': 3, 'weight_decay': 0.000302, 'patience': 6, 'clip_value': 83.631474} using model Sobol.


Number of parameters to train: 1097738


[INFO 09-30 09:52:35] ax.service.ax_client: Completed trial 0 with data: {'loss': (493.722241, None)}.
/usr/local/lib/python3.10/dist-packages/ax/modelbridge/cross_validation.py:463: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 09-30 09:52:35] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 0.000369, 'hidden_dim': 1024, 'no_layers': 5, 'weight_decay': 0.000648, 'patience': 4, 'clip_value': 41.444931} using model Sobol.


Number of parameters to train: 26816522


[INFO 09-30 09:58:12] ax.service.ax_client: Completed trial 1 with data: {'loss': (2505.885742, None)}.
/usr/local/lib/python3.10/dist-packages/ax/modelbridge/cross_validation.py:463: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 09-30 09:58:12] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 1e-06, 'hidden_dim': 256, 'no_layers': 4, 'weight_decay': 0.000196, 'patience': 1, 'clip_value': 22.814624} using model Sobol.


...

[INFO 09-30 12:04:46] ax.service.ax_client: Completed trial 47 with data: {'loss': (179.230341, None)}.
[INFO 09-30 12:04:49] ax.service.ax_client: Generated new trial 48 with parameters {'lr': 0.011824, 'hidden_dim': 1024, 'no_layers': 3, 'weight_decay': 1e-05, 'patience': 1, 'clip_value': 84.308101} using model BoTorch.


Number of parameters to train: 17367050


[INFO 09-30 12:08:04] ax.service.ax_client: Completed trial 48 with data: {'loss': (155.004431, None)}.
[INFO 09-30 12:08:10] ax.service.ax_client: Generated new trial 49 with parameters {'lr': 0.011379, 'hidden_dim': 1024, 'no_layers': 4, 'weight_decay': 0.001, 'patience': 10, 'clip_value': 43.320846} using model BoTorch.


Number of parameters to train: 22091786


[INFO 09-30 12:12:32] ax.service.ax_client: Completed trial 49 with data: {'loss': (215.310403, None)}.


In [ ]:
ax_client.get_trials_data_frame()

[WARNING 09-30 12:12:32] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


,trial_index,arm_name,trial_status,generation_method,loss,lr,hidden_dim,no_layers,weight_decay,patience,clip_value
0,0,0_0,COMPLETED,Sobol,493.722241,0.013386,256,3,0.000302,6,83.631474
1,1,1_0,COMPLETED,Sobol,2505.885742,0.000369,1024,5,0.000648,4,41.444931
2,2,2_0,COMPLETED,Sobol,5583.086914,0.000001,256,4,0.000196,1,22.814624
3,3,3_0,COMPLETED,Sobol,223.146011,0.029183,512,4,0.000871,9,63.709380
4,4,4_0,COMPLETED,Sobol,2309.062256,0.089323,512,5,0.000932,10,92.988826
5,5,5_0,COMPLETED,Sobol,5295.864355,0.000009,512,3,0.000014,2,39.490056
6,6,6_0,COMPLETED,Sobol,4969.165234,0.000100,256,5,0.000590,5,11.412602
7,7,7_0,COMPLETED,Sobol,795.276331,0.001619,1024,3,0.000487,7,64.720130
8,8,8_0,COMPLETED,Sobol,2648.862061,0.000656,512,4,0.000104,3,55.378168
9,9,9_0,COMPLETED,Sobol,4539.784668,0.000041,1024,3,0.000965,7,25.283368


In [ ]:
best_parameters, values = ax_client.get_best_parameters()
best_parameters

{'lr': 0.009595835793707791,
 'hidden_dim': 1024,
 'no_layers': 3,
 'weight_decay': 1e-05,
 'patience': 1,
 'clip_value': 5.1}

In [ ]:
render(ax_client.get_contour_plot(param_x="lr", param_y="weight_decay", metric_name="loss"))

[INFO 09-30 12:13:15] ax.service.ax_client: Retrieving contour plot with parameter 'lr' on X-axis and 'weight_decay' on Y-axis, for metric 'loss'. Remaining parameters are affixed to the middle of their range.


In [ ]:
render(
    ax_client.get_optimization_trace()
)